In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""
os.environ['AWS_DEFAULT_REGION'] = "ap-southeast-2"

In [2]:
import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/")
mlflow.set_experiment("Exp4_Handling_Imbalanced_Data")

<Experiment: artifact_location='s3://sentiment-mlops-bucket-s3/929218055700625099', creation_time=1761457708565, experiment_id='929218055700625099', last_update_time=1761457708565, lifecycle_stage='active', name='Exp4_Handling_Imbalanced_Data', tags={}>

In [3]:
import pandas as pd
df = pd.read_csv('cleaned_dataset.csv')


In [4]:
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [5]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def run_imbalances_experiment(imbalance_method):
    ngram_range = (1,2)  # Found out using the experiment earlier that we have to choose TFIDF and that too with bigram setting
    max_features = 1000 # Found out using the experiment earlier

    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], 
                                                        df['category'], 
                                                        test_size=0.2, 
                                                        random_state=42,
                                                        stratify=df['category']
                                                        )
    
    vectorizer = TfidfVectorizer(ngram_range=ngram_range,
                                 max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    if imbalance_method == 'class_weights':
        # use class weights in Random Forest
        class_weight = 'balanced'
    else:
        class_weight = None # Do not apply class weights if using resampling

        # Resampling techniques (only apply to training set)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state = 42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)
    
    with mlflow.start_run() as run:
        mlflow.set_tag('mlflow.runName', f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Bigrams")
        mlflow.set_tag('experiment_type', 'imbalance_handling')
        mlflow.set_tag('model_type', 'RandomForestClassifier')

        mlflow.set_tag('description', f'RandomForest with TFIDF bigrams, imbalance handling method = {imbalance_method}')

        # Log vectorization parameters
        mlflow.log_param("vectorizer_type", 'TFIDF')
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest params
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialise and train model
        model = RandomForestClassifier(n_estimators=n_estimators, 
                                       max_depth=max_depth,
                                       random_state=42,
                                       class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        y_pred_test = model.predict(X_test_vec)
        y_pred_train = model.predict(X_train_vec)

        # Log accuracy
        accuracy_score_train = accuracy_score(y_test, y_pred_test)
        accuracy_score_test = accuracy_score(y_train, y_pred_train)
        mlflow.log_metric("accuracy_score_test", accuracy_score_test)
        mlflow.log_metric("accuracy_score_train", accuracy_score_train)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred_test, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred_test)
        plt.figure(figsize = (8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt = 'd', cmap = 'Blues')
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TFIDF Bigrams, imbalance = {imbalance_method}")
        plt.savefig(f'confusion_matrix_{imbalance_method}.png')
        mlflow.log_artifact(f'confusion_matrix_{imbalance_method}.png')
        plt.close()

        # Log model
        mlflow.sklearn.log_model(model, 
                                 name = f"random_forest_model_tfidf_bigram_imbalance_{imbalance_method}",
                                 registered_model_name= f"random_forest_model_tfidf_bigram_imbalance_{imbalance_method}")
    

imbalance_method = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_method:
    run_imbalances_experiment(method)


2025/10/26 20:12:57 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/26 20:12:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'random_forest_model_tfidf_bigram_imbalance_class_weights'.
2025/10/26 20:13:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model_tfidf_bigram_imbalance_class_weights, version 1
Created version '1' of model 'random_forest_model_tfidf_bigram_imbalance_class_weights'.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Bigrams at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099/runs/bec338d033c045d8b58aed325bdc0677
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099


2025/10/26 20:13:32 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/26 20:13:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'random_forest_model_tfidf_bigram_imbalance_oversampling'.
2025/10/26 20:13:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model_tfidf_bigram_imbalance_oversampling, version 1
Created version '1' of model 'random_forest_model_tfidf_bigram_imbalance_oversampling'.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Bigrams at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099/runs/5a652b9651c641999d7c5d7021a09bf7
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099


2025/10/26 20:14:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/26 20:14:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'random_forest_model_tfidf_bigram_imbalance_adasyn'.
2025/10/26 20:14:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model_tfidf_bigram_imbalance_adasyn, version 1
Created version '1' of model 'random_forest_model_tfidf_bigram_imbalance_adasyn'.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Bigrams at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099/runs/d9b32e89ea4c4be988fc29ec70414f8d
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099


2025/10/26 20:15:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/26 20:15:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'random_forest_model_tfidf_bigram_imbalance_undersampling'.
2025/10/26 20:15:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model_tfidf_bigram_imbalance_undersampling, version 1
Created version '1' of model 'random_forest_model_tfidf_bigram_imbalance_undersampling'.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Bigrams at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099/runs/edb4fefd800840d4b31747210f46f726
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099


c:\Users\RadhikaMaheshwari\Downloads\mlops_sentiment_analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\RadhikaMaheshwari\Downloads\mlops_sentiment_analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\RadhikaMaheshwari\Downloads\mlops_sentiment_analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to

🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Bigrams at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099/runs/af985a0c863843ce8f95020b6d76ae8f
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/929218055700625099
